<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pull-flat,-distortion-calibs-for-JWST" data-toc-modified-id="Pull-flat,-distortion-calibs-for-JWST-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pull flat, distortion calibs for JWST</a></span></li><li><span><a href="#Set-context" data-toc-modified-id="Set-context-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set context</a></span><ul class="toc-item"><li><span><a href="#NIRISS" data-toc-modified-id="NIRISS-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>NIRISS</a></span></li><li><span><a href="#MIRI" data-toc-modified-id="MIRI-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>MIRI</a></span></li><li><span><a href="#NIRCam-SW" data-toc-modified-id="NIRCam-SW-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>NIRCam SW</a></span></li><li><span><a href="#NIRCam-LW" data-toc-modified-id="NIRCam-LW-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>NIRCam LW</a></span></li></ul></li></ul></div>

# Pull flat, distortion calibs for JWST

The idea here is to download and cache all of the basic calibration files for a specific CRDS_CONTEXT that are needed by `grizli` so that they don't need to be downloaded at runtime.

1. Set CRDS_CONTEXT
2. Loop over FILTER / PUPIL keywords for exposures of a given instrument
3. Run images through `grizli.prep.fresh_flt_file` to download and cache flat and distortion calibrations for that CRDS_CONTEXT


In [1]:
import os
import astropy.io.fits as pyfits

import jwst
from jwst.wfss_contam import WfssContamStep
from jwst.flatfield import FlatFieldStep
from jwst.assign_wcs import AssignWcsStep

from grizli import prep, utils


# Set context

In [2]:
os.environ['CRDS_CONTEXT'] = 'jwst_0916.pmap'

In [3]:
os.chdir('/Users/gbrammer/Research/JWST/PullCrdsContext')

_log = f"{os.environ['CRDS_CONTEXT']}.log"

if not os.path.exists(_log):
    with open(_log, 'w') as fp:
        fp.write('# inst filt pup\n')
        fp.write('inst filt pup\n')

if not os.path.exists('Orig'):
    os.system('aws s3 sync s3://grizli-v2/pull-crds-files/ Orig/')

! ls Orig/

jw01283001001_01101_00001_mirimage_cal.fits.gz
jw01324001001_01101_00003_nis_rate.fits.gz
jw01345011001_01101_00001_nrca5_rate.fits.gz
jw01433010001_01101_00005_nrca1_cal.fits.gz


## NIRISS

``flat``, ``distortion`` and ``wfssbkg`` references

In [4]:
from jwst.wfss_contam import WfssContamStep
        
log = utils.read_catalog(_log)
SKIP = True

file = 'jw01324001001_01101_00003_nis_rate.fits'
im = pyfits.open(f'Orig/{file}.gz')

for filt in ['CLEAR','GR150R','GR150C'][:]:
    for pup in ['F090W','F115W','F150W','F200W'][:]:
                
        ix = (log['inst'] == 'NIRISS') & (log['filt'] == filt) & (log['pup'] == pup)
        if (ix.sum() > 0) & SKIP:
            print('(skip) NIRISS', filt, pup)
            continue

        print('Run NIRISS', filt, pup)

        with open(_log, 'a') as fp:
            fp.write(f'NIRISS {filt} {pup}\n')

        im[0].header['FILTER'] = filt
        im[0].header['PUPIL'] = pup

        im.writeto(f'{file}', overwrite=True)
        
        #prep.fresh_flt_file(file, path='./Raw/')
        from jwst.flatfield import FlatFieldStep
        _flat =  FlatFieldStep().get_reference_file(file, 'flat')
        print('  flat:', _flat)

        if filt in ['GR150R','GR150C']:
            im = pyfits.open(file, mode='update')
            im[0].header['FILTER'] = filt
            im.flush()

            _ref =  WfssContamStep().get_reference_file(file, 'wfssbkg')
            print('  wfssbkg: ', _ref)
            
        _ref =  AssignWcsStep().get_reference_file(file, 'distortion')
        print('  distortion:', _ref)
        
        _ref =  AssignWcsStep().get_reference_file(file, 'filteroffset')
        print('  filteroffset:', _ref)


(skip) NIRISS CLEAR F090W
(skip) NIRISS CLEAR F115W
(skip) NIRISS CLEAR F150W
(skip) NIRISS CLEAR F200W
(skip) NIRISS GR150R F090W
(skip) NIRISS GR150R F115W
(skip) NIRISS GR150R F150W
(skip) NIRISS GR150R F200W
(skip) NIRISS GR150C F090W
(skip) NIRISS GR150C F115W
(skip) NIRISS GR150C F150W
(skip) NIRISS GR150C F200W


## MIRI

``flat``, ``distortion``, ``filteroffset`` references

In [5]:
# MIRI
from jwst.wfss_contam import WfssContamStep
        
log = utils.read_catalog(_log)
SKIP = True

file = 'jw01283001001_01101_00001_mirimage_cal.fits'
im = pyfits.open(f'Orig/{file}.gz')

pup = 'CLEAR'

for filt in ['F560W','F770W','F1000W','F1280W','F1500W','F1800W','F2100W'][:]:
                
        ix = (log['inst'] == 'MIRI') & (log['filt'] == filt) & (log['pup'] == pup)
        if (ix.sum() > 0) & SKIP:
            print('(skip) MIRI', filt, pup)
            continue

        print('Run MIRI', filt, pup)

        with open(_log, 'a') as fp:
            fp.write(f'MIRI {filt} {pup}\n')

        im[0].header['FILTER'] = filt
        #im[0].header['PUPIL'] = pup

        im.writeto(f'{file}', overwrite=True)
        
        #prep.fresh_flt_file(file, path='./Raw/')
        from jwst.flatfield import FlatFieldStep
        _flat =  FlatFieldStep().get_reference_file(file, 'flat')
        print('  flat:', _flat)
        
        _ref =  AssignWcsStep().get_reference_file(file, 'filteroffset')
        print('  filteroffset:', _ref)

        _ref =  AssignWcsStep().get_reference_file(file, 'distortion')
        print('  distortion:', _ref)

(skip) MIRI F560W CLEAR
(skip) MIRI F770W CLEAR
(skip) MIRI F1000W CLEAR
(skip) MIRI F1280W CLEAR
(skip) MIRI F1500W CLEAR
(skip) MIRI F1800W CLEAR
(skip) MIRI F2100W CLEAR


## NIRCam SW

In [6]:
# MIRI
from jwst.wfss_contam import WfssContamStep
        
log = utils.read_catalog(_log)
SKIP = True

file = 'jw01433010001_01101_00005_nrca1_cal.fits'
im = pyfits.open(f'Orig/{file}.gz')

pup = 'CLEAR'

for filt in ['F090W','F115W','F150W','F200W'][:]:
    for mod in 'AB':
        for det in range(4):
            pup = f'NRC{mod}{det+1}'
            
            ix = (log['inst'] == 'NIRCAM') & (log['filt'] == filt) & (log['pup'] == pup)
            if (ix.sum() > 0) & SKIP:
                print('(skip) NIRCAM', filt, pup)
                continue

            print('Run NIRCAM', filt, pup)

            with open(_log, 'a') as fp:
                fp.write(f'NIRCAM {filt} {pup}\n')

            im[0].header['FILTER'] = filt
            im[0].header['DETECTOR'] = pup
            im[0].header['MODULE'] = mod
            
            im.writeto(f'{file}', overwrite=True)

            #prep.fresh_flt_file(file, path='./Raw/')
            from jwst.flatfield import FlatFieldStep
            _flat =  FlatFieldStep().get_reference_file(file, 'flat')
            print('  flat:', _flat)
            
            _ref =  AssignWcsStep().get_reference_file(file, 'filteroffset')
            print('  filteroffset:', _ref)

            _ref =  AssignWcsStep().get_reference_file(file, 'distortion')
            print('  distortion:', _ref)

(skip) NIRCAM F090W NRCA1
(skip) NIRCAM F090W NRCA2
(skip) NIRCAM F090W NRCA3
(skip) NIRCAM F090W NRCA4
(skip) NIRCAM F090W NRCB1
(skip) NIRCAM F090W NRCB2
(skip) NIRCAM F090W NRCB3
(skip) NIRCAM F090W NRCB4
(skip) NIRCAM F115W NRCA1
(skip) NIRCAM F115W NRCA2
(skip) NIRCAM F115W NRCA3
(skip) NIRCAM F115W NRCA4
(skip) NIRCAM F115W NRCB1
(skip) NIRCAM F115W NRCB2
(skip) NIRCAM F115W NRCB3
(skip) NIRCAM F115W NRCB4
(skip) NIRCAM F150W NRCA1
(skip) NIRCAM F150W NRCA2
(skip) NIRCAM F150W NRCA3
(skip) NIRCAM F150W NRCA4
(skip) NIRCAM F150W NRCB1
(skip) NIRCAM F150W NRCB2
(skip) NIRCAM F150W NRCB3
(skip) NIRCAM F150W NRCB4
(skip) NIRCAM F200W NRCA1
(skip) NIRCAM F200W NRCA2
(skip) NIRCAM F200W NRCA3
(skip) NIRCAM F200W NRCA4
(skip) NIRCAM F200W NRCB1
(skip) NIRCAM F200W NRCB2
(skip) NIRCAM F200W NRCB3
(skip) NIRCAM F200W NRCB4


## NIRCam LW

``flat``, ``distortion``, ``filteroffset``, and ``wfssbkg`` for slitless.

In [7]:
# MIRI
from jwst.wfss_contam import WfssContamStep
        
log = utils.read_catalog(_log)
SKIP = True

file = 'jw01345011001_01101_00001_nrca5_rate.fits'
im = pyfits.open(f'Orig/{file}.gz')

pup = 'CLEAR'

for filt in ['F277W','F356W','F410M','F444W'][:]:
    for mod in 'AB':
        for pup in ['CLEAR','GRISMR','GRISMC']:
            key = f'NRC{mod}LONG-{pup}'
            
            ix = (log['inst'] == 'NIRCAM') & (log['filt'] == filt) & (log['pup'] == key)
            if (ix.sum() > 0) & SKIP:
                print('(skip) NIRCAM', filt, key)
                continue

            print('Run NIRCAM', filt, key)

            with open(_log, 'a') as fp:
                fp.write(f'NIRCAM {filt} {key}\n')

            im[0].header['FILTER'] = filt
            im[0].header['DETECTOR'] = f'NRC{mod}LONG'
            im[0].header['MODULE'] = mod
            im[0].header['PUPIL'] = pup
            
            im.writeto(f'{file}', overwrite=True)
            
            #prep.fresh_flt_file(file, path='./Raw/')
            from jwst.flatfield import FlatFieldStep
            _flat =  FlatFieldStep().get_reference_file(file, 'flat')
            print('  flat:', _flat)

            _ref =  AssignWcsStep().get_reference_file(file, 'distortion')
            print('  distortion:', _ref)
            
            _ref =  AssignWcsStep().get_reference_file(file, 'filteroffset')
            print('  filteroffset:', _ref)

            if pup in ['GRISMR','GRISMC']:
                _ref =  WfssContamStep().get_reference_file(file, 'wfssbkg')
                print('  wfssbkg: ', _ref)
                

(skip) NIRCAM F277W NRCALONG-CLEAR
(skip) NIRCAM F277W NRCALONG-GRISMR
(skip) NIRCAM F277W NRCALONG-GRISMC
(skip) NIRCAM F277W NRCBLONG-CLEAR
(skip) NIRCAM F277W NRCBLONG-GRISMR
(skip) NIRCAM F277W NRCBLONG-GRISMC
(skip) NIRCAM F356W NRCALONG-CLEAR
(skip) NIRCAM F356W NRCALONG-GRISMR
(skip) NIRCAM F356W NRCALONG-GRISMC
(skip) NIRCAM F356W NRCBLONG-CLEAR
(skip) NIRCAM F356W NRCBLONG-GRISMR
(skip) NIRCAM F356W NRCBLONG-GRISMC
(skip) NIRCAM F410M NRCALONG-CLEAR
(skip) NIRCAM F410M NRCALONG-GRISMR
(skip) NIRCAM F410M NRCALONG-GRISMC
(skip) NIRCAM F410M NRCBLONG-CLEAR
(skip) NIRCAM F410M NRCBLONG-GRISMR
(skip) NIRCAM F410M NRCBLONG-GRISMC
(skip) NIRCAM F444W NRCALONG-CLEAR
(skip) NIRCAM F444W NRCALONG-GRISMR
(skip) NIRCAM F444W NRCALONG-GRISMC
(skip) NIRCAM F444W NRCBLONG-CLEAR
(skip) NIRCAM F444W NRCBLONG-GRISMR
(skip) NIRCAM F444W NRCBLONG-GRISMC
